# Dyck Task - Inference (Google Colab)

Run inference with your fine-tuned model. Upload `results/` or set `MODEL_PATH` to that folder.

In [ ]:
# Optional: !pip install -q torch transformers unsloth bitsandbytes

In [ ]:
import torch
import json
from unsloth import FastLanguageModel

MODEL_PATH = "results"
MAX_LENGTH = 2048
OUTPUT_PATH = "inference_results.jsonl"
BRACKET_PAIRS = [("(", ")"), ("[", "]"), ("{", "}"), ("<", ">"), ("⟨", "⟩"), ("⟦", "⟧"), ("⦃", "⦄"), ("⦅", "⦆")]

In [ ]:
print("Loading model...")
model, tokenizer = FastLanguageModel.from_pretrained(model_name=MODEL_PATH, max_seq_length=MAX_LENGTH, dtype=torch.float16, load_in_4bit=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
FastLanguageModel.for_inference(model)
print("Model loaded.")

In [ ]:
sequence = "<[<⟨{(<[(("
bracket_list = ", ".join(f"{o}/{c}" for o, c in BRACKET_PAIRS)
prompt = f"Bracket pairs (opening/closing): {bracket_list}. Each symbol is a single bracket (e.g. ⟦ is opening, ⟧ is closing).\nComplete the following Dyck sequence: {sequence}\nEnd with FINAL ANSWER: then only the closing brackets."
messages = [{"role": "user", "content": prompt}]
user_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
user_tokenized = tokenizer(user_text, truncation=True, max_length=MAX_LENGTH, return_tensors="pt", padding=True, add_special_tokens=True)
input_ids = user_tokenized["input_ids"].to(model.device)
prompt_length = input_ids.shape[1]
output_ids = model.generate(input_ids, max_new_tokens=1024, do_sample=True, temperature=0.1, top_p=0.9, top_k=10, repetition_penalty=1.25, pad_token_id=tokenizer.pad_token_id)
generated_ids = output_ids[0][prompt_length:]
response = tokenizer.decode(generated_ids, skip_special_tokens=True)
if "FINAL ANSWER:" in response:
    response = response.split("FINAL ANSWER:")[-1].strip().split("\n")[0].strip()
print(response)

In [ ]:
with open(OUTPUT_PATH, 'a', encoding='utf-8') as f:
    f.write(json.dumps({"sequence": sequence, "response": response}, ensure_ascii=False) + '\n')
print(f"Saved to {OUTPUT_PATH}")